In [1]:
import json
from pathlib import Path
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
from dataclasses import dataclass, asdict
from itertools import batched

In [2]:
DATA_PATH = Path("../data/processed/chunked_notes/chunked_notes.json")
BATCH_SIZE = 128
COLLECTION_NAME = "chunked_notes"
CHROMA_HOST = "localhost"
CHROMA_PORT = 8000
MODEL_NAME = "all-MiniLM-L6-v2"

## Wczytanie danych

In [4]:
@dataclass(frozen=True)
class DocumentChunk:
    id: str
    content: str
    filename: str
    course: str
    title: str
    other_metadata: dict[str, str]

    def metadata_dict(self):
        return {
            "filename": self.filename,
            "course": self.course,
            "title": self.title,
            **self.other_metadata,
        }


In [5]:
def load_items(path: Path):
    with open(path, "r") as f:
        raw_items = json.load(f)
    return [
        DocumentChunk(**raw_item)
        for raw_item in raw_items
    ]

items = load_items(DATA_PATH)
len(items)

10501

In [6]:
items[0]

DocumentChunk(id='7e8b362d-12cd-4139-8966-ca913ca10240', content='# Transformacja cyfrowa a wdrażanie i utrzymywanie systemów (2025-05-29)  \n## Transformacja cyfrowa\n* Cyfrowa transformacja to nie to samo co cyfryzacja / digitalizacja\n* Digitalizacja\n* zastąpienie części procesu biznesowego oprogramowaniem', filename='wyklad-12', course='wdrazanie-i-utrzymywanie-systemow-informatycznych', title='Transformacja cyfrowa a wdrażanie i utrzymywanie systemów (2025-05-29)', other_metadata={'H1': 'Transformacja cyfrowa a wdrażanie i utrzymywanie systemów (2025-05-29)', 'H2': 'Transformacja cyfrowa'})

## Klient ChromaDB

In [7]:
client = chromadb.HttpClient(
    host=CHROMA_HOST,
    port=CHROMA_PORT,
)

In [8]:
client.delete_collection(COLLECTION_NAME)

In [9]:
collection = client.get_or_create_collection(name=COLLECTION_NAME)

In [10]:
client.list_collections()

[Collection(name=chunked_notes)]

In [11]:
collection.count()

0

In [12]:
batch = items[:10]
docs = [item.content for item in batch]
ids = [item.id for item in batch]
metadatas = [item.metadata_dict() for item in batch]

# len(embeddings[0])
collection.add(
    ids=ids,
    documents=docs,
    metadatas=metadatas,
)

2025-11-06 16:04:58.432617401 [W:onnxruntime:Default, device_discovery.cc:164 DiscoverDevicesForPlatform] GPU device discovery failed: device_discovery.cc:89 ReadFileContents Failed to open file: "/sys/class/drm/card0/device/vendor"


In [13]:
collection.count()

10

In [14]:
collection.get(ids=[ids[0]])

{'ids': ['7e8b362d-12cd-4139-8966-ca913ca10240'],
 'embeddings': None,
 'metadatas': [{'filename': 'wyklad-12',
   'title': 'Transformacja cyfrowa a wdrażanie i utrzymywanie systemów (2025-05-29)',
   'course': 'wdrazanie-i-utrzymywanie-systemow-informatycznych',
   'H2': 'Transformacja cyfrowa',
   'H1': 'Transformacja cyfrowa a wdrażanie i utrzymywanie systemów (2025-05-29)'}],
 'documents': ['# Transformacja cyfrowa a wdrażanie i utrzymywanie systemów (2025-05-29)  \n## Transformacja cyfrowa\n* Cyfrowa transformacja to nie to samo co cyfryzacja / digitalizacja\n* Digitalizacja\n* zastąpienie części procesu biznesowego oprogramowaniem'],
 'data': None,
 'uris': None,
 'included': ['metadatas', 'documents']}

In [15]:
collection.get(include=["documents", "metadatas", "embeddings"])

{'ids': ['7e8b362d-12cd-4139-8966-ca913ca10240',
  '2c714bda-ea47-4be1-aeda-ae46fdda7308',
  '620850d6-980a-45e1-98ad-adaa0dfc6b55',
  'ce7c0d5a-5d50-45b1-9ead-c8fc4903a8c1',
  '80c220fe-5e8b-4a11-8740-0e8e2ccf34f8',
  '1daf0c64-d609-4c59-a0a6-e2b5b46ff7ca',
  '53a9d82a-2c04-418e-a79e-6bf229662667',
  'b1101c04-a416-497a-9045-9d3d0a0309b7',
  '98a9ca64-898f-469c-945e-57b82f066a01',
  '33231f00-869a-422b-8c16-80a5af37f0ed'],
 'embeddings': array([[-0.09890205,  0.05635744, -0.04719115, ...,  0.02970754,
          0.01138219, -0.03179566],
        [-0.0833109 ,  0.07807598,  0.01197425, ...,  0.03891951,
          0.13646966, -0.08573389],
        [-0.13168266,  0.01924606, -0.04312998, ...,  0.01423862,
          0.01297054,  0.00684466],
        ...,
        [ 0.02171782,  0.07962877,  0.02420806, ...,  0.03744587,
          0.03924099, -0.05432119],
        [-0.03921131,  0.14549549, -0.06473972, ...,  0.02907859,
          0.11127786, -0.07143361],
        [-0.03984833,  0.11301908, 

## Zasilenie ChromaDB danymi

In [21]:
client.delete_collection(COLLECTION_NAME)
collection = client.get_or_create_collection(name=COLLECTION_NAME)

for idx, batch in enumerate(batched(items, BATCH_SIZE)):
    print("Adding batch", idx)
    docs = [item.content for item in batch]
    ids = [item.id for item in batch]
    metadatas = [item.metadata_dict() for item in batch]

    collection.add(
        ids=ids,
        documents=docs,
        metadatas=metadatas,
    )

Adding batch 0
Adding batch 1
Adding batch 2
Adding batch 3
Adding batch 4
Adding batch 5
Adding batch 6
Adding batch 7
Adding batch 8
Adding batch 9
Adding batch 10
Adding batch 11
Adding batch 12
Adding batch 13
Adding batch 14
Adding batch 15
Adding batch 16
Adding batch 17
Adding batch 18
Adding batch 19
Adding batch 20
Adding batch 21
Adding batch 22
Adding batch 23
Adding batch 24
Adding batch 25
Adding batch 26
Adding batch 27
Adding batch 28
Adding batch 29
Adding batch 30
Adding batch 31
Adding batch 32
Adding batch 33
Adding batch 34
Adding batch 35
Adding batch 36
Adding batch 37
Adding batch 38
Adding batch 39
Adding batch 40
Adding batch 41
Adding batch 42
Adding batch 43
Adding batch 44
Adding batch 45
Adding batch 46
Adding batch 47
Adding batch 48
Adding batch 49
Adding batch 50
Adding batch 51
Adding batch 52
Adding batch 53
Adding batch 54
Adding batch 55
Adding batch 56
Adding batch 57
Adding batch 58
Adding batch 59
Adding batch 60
Adding batch 61
Adding batch 62
Ad

## Przykładowe zapytanie

In [22]:
collection.query(query_texts=["system operacyjny"], n_results=3)

{'ids': [['1625da67-d199-4b02-a8a5-be23b472e3e3',
   '4482d08d-d5ee-46f2-8de5-d7dfc35b6f3c',
   '68d88add-f310-46a8-9fa5-dbd4e8afc7ed']],
 'distances': [[0.7405853, 0.7587714, 0.7767137]],
 'embeddings': None,
 'metadatas': [[{'course': 'bezpieczenstwo-systemow-i-sieci',
    'title': 'Wstęp',
    'filename': '01-wstep',
    'H1': 'Wstęp',
    'H2': 'Bezpieczeństwo systemu IoT',
    'H3': 'Budowa systemu'},
   {'H2': 'Program w systemie operacyjnym',
    'H1': 'Wykład 09 (2023-03-22)',
    'course': 'architektura-komputerow',
    'filename': 'wyklad-09-konsolidacja',
    'title': 'Wykład 09 (2023-03-22)'},
   {'filename': 'kolokwium-01',
    'title': 'Kolokwium 1',
    'H3': 'Z systemem operacyjnym',
    'course': 'systemy-komputerowe-w-sterowaniu-i-pomiarach',
    'H2': 'Różne podejścia do tworzenia oprogramowania dla systemów wbudowanych',
    'H1': 'Kolokwium 1'}]],
 'documents': [['## Bezpieczeństwo systemu IoT  \n### Budowa systemu\n* Sprzęt\n* System operacyjny (Linux)\n* UI (najc